#  주간 음악 순위
URL :: https://music.naver.com/listen/history/index.nhn?type=TOTAL&year=2018&month=11&week=1

TOP 100 : 순위, 곡명, 아티스트

In [96]:
import glob
import time
from datetime import datetime

import os
from os.path import exists

import requests
from bs4 import BeautifulSoup

In [97]:
url = 'https://music.naver.com/listen/history/index.nhn?type=TOTAL_V2&year=2019&month=08&week=0'

html_music = requests.get(url).text
soup_music = BeautifulSoup(html_music, "lxml")
soup_music.title

<title>TOP 100 차트 히스토리 : 네이버 뮤직</title>

In [98]:
# _title 정보


## a 태그의 요소 중에서 class 속성값이 "_title" 인 것을 찾고
# 그 안에서 span 태그의 요소 중에서 class 속성값이 "ellipsis"인 요소를 추출

#  <span class="ellipsis">그대라는 시</span>



titles = soup_music.select('a._title span.ellipsis') 
len(titles)

50

In [132]:
t = soup_music.select('td.change')


In [137]:
t[0].get_text().split('\n')[2]

'61'

In [100]:
titles[0:10]

[<span class="ellipsis">그대라는 시</span>,
 <span class="ellipsis">술이 문제야</span>,
 <span class="ellipsis">ICY</span>,
 <span class="ellipsis">헤어져줘서 고마워</span>,
 <span class="ellipsis">기억해줘요 내 모든 날과 그때를</span>,
 <span class="ellipsis">2002</span>,
 <span class="ellipsis">나의 어깨에 기대어요</span>,
 <span class="ellipsis">니 소식</span>,
 <span class="ellipsis">사랑에 연습이 있었다면 (Prod. 2soo)</span>,
 <span class="ellipsis">Snapping</span>]

In [101]:
music_titles = [title.get_text() for title in titles]
music_titles[0:10]

['그대라는 시',
 '술이 문제야',
 'ICY',
 '헤어져줘서 고마워',
 '기억해줘요 내 모든 날과 그때를',
 '2002',
 '나의 어깨에 기대어요',
 '니 소식',
 '사랑에 연습이 있었다면 (Prod. 2soo)',
 'Snapping']

In [102]:
# _artist 정보

# a 태그의 요소 중에서 class 속성값이 "_artist" 인 것을 찾고
# 그 안에서 span 태그의 요소 중에서 class 속성값이 "ellipsis"인 요소를 추출


#<a href="/artist/home.nhn?artistId=35551" class="_artist NPI=a:artist,r:1,i:35551" title="태연 (TAEYEON)">
# <span class="ellipsis">태연 (TAEYEON)</span>
#</a>


artists = soup_music.select('a._artist span.ellipsis')

In [103]:
artists[0].get_text()

'\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t태연 (TAEYEON)\r\n\t\t'

In [105]:
artists[0].get_text().strip()

'태연 (TAEYEON)'

In [107]:
music_artists = [artist.get_text().strip() for artist in artists]
music_artists[0:10]

['태연 (TAEYEON)',
 'ITZY(있지)',
 '벤',
 '거미',
 'Anne-Marie',
 '10cm',
 '송하예',
 '임재현',
 '청하',
 '헤이즈 (Heize)']

In [108]:
#합치기
music_titles_artists={}
ranking = 0

for (music_title, music_artist) in zip(music_titles, music_artists):
    ranking += 1
    music_titles_artists[ranking] = [music_title, music_artist]
    
    
music_titles_artists[10]

['Snapping', '헤이즈 (Heize)']

In [110]:
#월 구하기
now = datetime.now()
month = now.month
month = 12




month = str(1) if not str(month).isdigit() else str(month)
#*.isdigit() 문자가 숫자인지 판단
month = '0'+month if len(month)==1 else month
month

'12'

In [113]:
#내용

# url = 'https://music.naver.com/listen/history/index.nhn?type=OVERSEA_V2&year=2019&month=08&week=4'
# url = 'https://music.naver.com/listen/history/index.nhn?type=DOMESTIC_V2&year=2019&month=08&week=4'
# url = 'https://music.naver.com/listen/history/index.nhn?type=TOTAL_V2&year=2019&month=08&week=0'

type_dict = { '종합' : 'TOTAL_V2', 
              '국내' : 'DOMESTIC_V2', 
              '해외' : 'OVERSEA_V2'  } 

url = 'https://music.naver.com/listen/history/index.nhn?type={type}&year={year}&month={month}&week={week}'.format(
            type  = type_dict['국내'], 
            year  = '2019',
            month = '08',
            week  = '0' 
       )

html_music = requests.get(url).text
soup_music = BeautifulSoup(html_music, "lxml")

titles  = soup_music.select('a._title span.ellipsis') 
artists = soup_music.select('td._artist a')

music_titles  = [title.get_text() for title in titles]
music_artists = [artist.get_text().strip() for artist in artists]

for k in range(10):
    print("{0}: {1} / {2}".format(k+1, music_titles[k], music_artists[k]))

1: 그대라는 시 / 태연 (TAEYEON)
2: 술이 문제야 / 장혜진
3: ICY / ITZY(있지)
4: 헤어져줘서 고마워 / 벤
5: 기억해줘요 내 모든 날과 그때를 / 거미
6: 나의 어깨에 기대어요 / 10cm
7: 니 소식 / 송하예
8: 사랑에 연습이 있었다면 (Prod. 2soo) / 임재현
9: Snapping / 청하
10: 작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey) / 방탄소년단


In [114]:
#모듈화

type_dict = { '종합' : 'TOTAL_V2', 
              '국내' : 'DOMESTIC_V2', 
              '해외' : 'OVERSEA_V2'  } 

# 네이버 music 주소를 입력하면 노래 제목과 아티스트를 반환
# def naver_music(url):
def naver_music(type='종합', year=2019, month=1, week=0, page=1):
    
    now = datetime.now()
    year  = str(now.year) if not str(year).isdigit() else str(year)
    month = str(1) if not str(month).isdigit() else str(month)
    month = '0'+month if len(month)==1 else month
    week  = str(0) if not str(week).isdigit() else str(week)
    
    url = 'https://music.naver.com/listen/history/index.nhn?type={type}&year={year}&month={month}&week={week}&page={page}'
    url =  url.format(
            type  = type_dict['국내'], 
            year  = year,
            month = month,
            week  = week,
            page  = page
       )
    print(url)
    
    html_music = requests.get(url).text
    soup_music = BeautifulSoup(html_music, "lxml")

    titles = soup_music.select('a._title span.ellipsis') 
    artists = soup_music.select('td._artist a')

    music_titles  = [ title.get_text().strip()  for title  in titles  ]
    music_artists = [ artist.get_text().strip() for artist in artists ]
    
    return music_titles, music_artists

In [116]:
##저장하기

# 노래 제목과 아티스트를 저장할 파일 이름을 폴더와 함께 지정
file_name = './data/NaverMusicTop100.txt'
if exists(file_name): os.remove(file_name)

f = open(file_name,'w') # 파일 열기

# 각 page에는 50개의 노래 제목과 아티스트가 추출됨
for page in [1, 2]:
    # naver_music_url_page = naver_music_url + str(page+1)     # page URL
    # nave_music_titles, naver_music_artists = naver_music(naver_music_url_page)
    nave_music_titles, naver_music_artists = naver_music(type='종합', 
                                                         year=2019, month=8, week=0,
                                                         page=page)
    
    # 추출된 노래 제목과 아티스트를 파일에 저장 
    for idx in range(len(music_titles_artists)):
        ranking = (page-1)*50 + idx + 1 
        f.write("{0:3d}: {1}/{2}\n".format(ranking,
                                           nave_music_titles[idx],  
                                           naver_music_artists[idx]))
    time.sleep(1)

f.close() # 파일 닫기

glob.glob(file_name) # 생성된 파일 확인

https://music.naver.com/listen/history/index.nhn?type=DOMESTIC_V2&year=2019&month=08&week=0&page=1
https://music.naver.com/listen/history/index.nhn?type=DOMESTIC_V2&year=2019&month=08&week=0&page=2


['./data/NaverMusicTop100.txt']

In [117]:
!type .\data\NaverMusicTop100.txt

  1: 그대라는 시/태연 (TAEYEON)
  2: 술이 문제야/장혜진
  3: ICY/ITZY(있지)
  4: 헤어져줘서 고마워/벤
  5: 기억해줘요 내 모든 날과 그때를/거미
  6: 나의 어깨에 기대어요/10cm
  7: 니 소식/송하예
  8: 사랑에 연습이 있었다면 (Prod. 2soo)/임재현
  9: Snapping/청하
 10: 작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)/방탄소년단
 11: 내 맘을 볼 수 있나요/헤이즈 (Heize)
 12: 오늘도 빛나는 너에게 (To You My Light) (Feat. 이라온)/마크툽(Maktub)
 13: 포장마차/황인욱
 14: 너에게 못했던 내 마지막 말은/다비치
 15: 솔직하게 말해서 나/김나영
 16: 비가 내리는 날에는/윤하
 17: 안녕/폴킴
 18: Another Day/먼데이키즈(Mon..
 19: FANCY/TWICE(트와이스)
 20: 사계 (Four Seasons)/태연 (TAEYEON)
 21: 인사/멜로망스(Melomance)
 22: 너를 만나/폴킴
 23: 열대야 (Fever)/여자친구(GFRIEND)
 24: 달라달라/ITZY(있지)
 25: 벌써 12시/청하
 26: 모든 날, 모든 순간 (Every day, Every Moment)/폴킴
 27: Goodbye/박효신
 28: 그 끝에 그대/청하
 29: 짐살라빔 (Zimzalabim)/Red Velvet (레드벨벳)
 30: 너만 너만 너만/양다일
 31: 그때가 좋았어/케이시(Kassy)
 32: 주저하는 연인들을 위해/잔나비
 33: We don't talk together (Feat. 기리보이) (Prod. SUGA)/헤이즈 (Heize)
 34: 여행/볼빨간사춘기
 35: 내 목소리 들리니/벤
 36: BAND/창모(CHANGM..
 37: BIRTHDAY/전소미
 38: 사랑이 식었다고 말해도 돼/먼데이키즈(Monday Kiz)
 39: 노래방에서/장범준
 40: Dance T